In [1]:
"""
Final Submission Notebook for CMI Kaggle Competition.
This notebook integrates the trained model and the finalized feature engineering
function within the required Kaggle API loop structure.
"""

import pandas as pd
import numpy as np
import data.kaggle_evaluation
import os
import pickle
import joblib
import catboost as cat

In [2]:
from src.config import PROJECT_PATH, DATA_PATH
from src.feature_engineering import create_wave0_features
TEST_DATA_FILE = 'test.csv'
TEST_DEMOGRAPHICS_FILE = 'test_demographics.csv'

In [20]:
# --- Update these paths to where you save your final model and feature function ---
MODEL_PATH = 'models_rev/wave1-catboost-best.cbm'
FEATURE_FUNCTION_NAME = 'create_wave0_features' # Name of the final function

In [4]:
#GLOBAL MAPS
# Load training data
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_train_demos = pd.read_csv(os.path.join(DATA_PATH, 'train_demographics.csv'))

# --- Create Helper Mappings for Evaluation Metric ---
# Important for the custom F1 score function
metadata = df_train[['gesture', 'sequence_type']].drop_duplicates()

# Map gesture string to sequence type (Target vs. Non-Target)
gesture_to_seq_type_map = metadata.set_index('gesture')['sequence_type'].to_dict()

# Map gesture string to integer code and back
gesture_map = {label: i for i, label in enumerate(metadata['gesture'].unique())}
inv_gesture_map = {i: label for label, i in gesture_map.items()}

# Validate
print(f"Gesture Map: {gesture_map}")
print(f"\nInverted Gesture Map: {inv_gesture_map}")
print(f"\nGesture To Sequence Type: {gesture_to_seq_type_map}")

Gesture Map: {'Cheek - pinch skin': 0, 'Forehead - pull hairline': 1, 'Write name on leg': 2, 'Feel around in tray and pull out an object': 3, 'Neck - scratch': 4, 'Neck - pinch skin': 5, 'Eyelash - pull hair': 6, 'Eyebrow - pull hair': 7, 'Forehead - scratch': 8, 'Above ear - pull hair': 9, 'Wave hello': 10, 'Write name in air': 11, 'Text on phone': 12, 'Pull air toward your face': 13, 'Pinch knee/leg skin': 14, 'Scratch knee/leg skin': 15, 'Drink from bottle/cup': 16, 'Glasses on/off': 17}

Inverted Gesture Map: {0: 'Cheek - pinch skin', 1: 'Forehead - pull hairline', 2: 'Write name on leg', 3: 'Feel around in tray and pull out an object', 4: 'Neck - scratch', 5: 'Neck - pinch skin', 6: 'Eyelash - pull hair', 7: 'Eyebrow - pull hair', 8: 'Forehead - scratch', 9: 'Above ear - pull hair', 10: 'Wave hello', 11: 'Write name in air', 12: 'Text on phone', 13: 'Pull air toward your face', 14: 'Pinch knee/leg skin', 15: 'Scratch knee/leg skin', 16: 'Drink from bottle/cup', 17: 'Glasses on/of

## Load model and feature function

In [22]:
model = cat.CatBoostClassifier()
model.load_model(MODEL_PATH);

In [23]:
model